In [1]:
# Do not delete this cell. It ensures that you can do the imports,
# load datasets etc. in the same fashion as in any Python script
# in the project template.


import sys
sys.path.insert(0, '../..')
from bld.project_paths import project_paths_join as ppj
from bld.project_paths import project_paths as pp



In [2]:
import numpy as np
import json
import pickle
import pandas as pd
from scipy.stats import sem
import seaborn as sns
import matplotlib.pyplot as plt
%config Completer.use_jedi = False

from matplotlib import rc
rc('text', usetex=True)


In [7]:
with open(
    ppj("IN_DATA", f"session_info_experiments.json")
) as f:
    SESSION_INFO = json.load(f)


In [102]:
data_list = []
for session_id, session_info in SESSION_INFO.items():
    current_data = pd.read_csv(ppj("IN_DATA", session_info['path']))
    current_data['session_id'] = session_id
    data_list.append(current_data)
data_all = pd.concat(data_list)

In [218]:
(data_all['participant.payoff'] / 130 + 4).mean()

11.395201129146084

In [131]:
info_dict = {
    'algos_to_draw':
        {
            '1H1A': 32,
            '1H2A': 42,
            '2H1A': 21,
        },
    'relevant_payoff_field_by_sg':
        {
            1: 'bertrand.25.group.accumulated_profit_algorithm',
            2: 'bertrand_SG_2.17.group.accumulated_profit_algorithm',
            3: 'bertrand_SG_3.11.group.accumulated_profit_algorithm'
        },
    'grouping_field_by_sg':
    {
        1: 'bertrand.1.group.id_in_subsession',
        2: 'bertrand_SG_2.1.group.id_in_subsession',
        3: 'bertrand_SG_3.1.group.id_in_subsession',

    }
}

In [208]:
# Draw the super games that are paid for each treatment
np.random.seed(267) # Todays day as a random seed
sg_paid = np.random.randint(1,4,3)
treatments_list = ['1H1A', '1H2A', '2H1A']


In [228]:
all_payoffs = []

for treatment, paid_sg in zip(treatments_list, sg_paid):
    relevant_subset = data_all.loc[data_all['bertrand.1.group.group_treatment'] == treatment].copy()
    relevant_payment_data = data_all.loc[
                                data_all['bertrand.1.group.group_treatment'] == treatment
                            ].groupby(
                                ['session_id', info_dict['grouping_field_by_sg'][paid_sg]]
                            )[info_dict['relevant_payoff_field_by_sg'][paid_sg]].mean().values.flatten()

    if treatment == '1H1A':
        assert len(relevant_subset) == info_dict['algos_to_draw'][treatment]
        all_payoffs.append(relevant_payment_data)
    elif treatment == '1H2A':
        assert len(relevant_subset) == info_dict['algos_to_draw'][treatment] / 2
        # In the 1H2A treatment, each values has to be repeated twice as there are two 
        # identical algos in each market 
        all_payoffs.append(np.repeat(relevant_payment_data, 2))

    elif treatment == '2H1A':
        assert len(relevant_subset) == info_dict['algos_to_draw'][treatment] * 2
        # Here for each group there exists only on algorithm
        # Note that the way how we create the *relevant_payment_data* object
        # already accounts for that.
        all_payoffs.append(relevant_payment_data)
        
all_payoffs_with_show_up_euro = (np.concatenate(all_payoffs) / 130 + 4).round(2)


In [231]:
pd.DataFrame(all_payoffs_with_show_up_euro, columns=['payments'])

,payments
0,5.85
1,19.69
2,8.62
3,19.69
4,19.69
...,...
90,6.08
91,8.69
92,7.08
93,7.15


In [221]:
with open(ppj("OUT_DATA", "data_individual_level.pickle"), "rb") as f:
    data_individual_level = pickle.load(f)

In [226]:
data_individual_level.loc[(data_individual_level['super_game'] == 1) &
                          (data_individual_level['round'] == 1)].groupby(['treatment'])['participant.code'].count().to_dict()

{'1H1A': 32, '1H2A': 21, '2H0A': 60, '2H1A': 42, '3H0A': 63}